In [1]:
import time
import numpy as np

In [2]:
import mosaic
from mosaic import tessera

Mosaic is an actor-based parallelisation library. Actors, the basic unit of parallelism in mosaic, are represented by classes called `tessera`.

A `tessera` can be created by decorating any Python class with the `@tessera` decorator:

In [3]:
@tessera
class Solver1:
    def __init__(self, data):
        self.data = data

    def solve(self, data):
        print('Solve 1')
        self.data = self.data + data

        time.sleep(10)
        print('Done 1')

        return self.data

    def solve_more(self):
        print('Solve More 1')
        time.sleep(5)
        print('Done More 1')


@tessera
class Solver2:
    def __init__(self):
        self.data = 0

    def solve(self, data):
        print('Solve 2')
        self.data = data*2

        time.sleep(10)
        print('Done 2')

        return self.data

    def solve_more(self):
        print('Solve More 2')
        time.sleep(5)
        print('Done More 2')

Before we start working with the `tessera` classes that we have just created, we need to start the mosaic runtime that will manage them. There are several ways to start the runtime but, because we are working with a Jupyter notebook, we want to start it in interactive mode:

In [4]:
await mosaic.interactive('on', num_workers=2)

HEAD            Listening at <CommsManager object at 140231089546320, uid=head, address=155.198.98.57, port=3000, state=listening>
MONITOR         Listening at <CommsManager object at 140229011265808, uid=monitor, address=155.198.98.57, port=3001, state=listening>
NODE:0          Listening at <CommsManager object at 140230615021648, uid=node:0, address=155.198.98.57, port=3003, state=listening>
WORKER:0:0      Listening at <CommsManager object at 140230615043472, uid=worker:0:0, address=155.198.98.57, port=3006, state=listening>
WORKER:0:1      Listening at <CommsManager object at 140230615032208, uid=worker:0:1, address=155.198.98.57, port=3008, state=listening>


Now, we can start using mosaic's runtime to execute our parallel workload. To do that, let's instantiate some of our `tessera` by calling the `remote` method that is now available for each of our classes:

In [5]:
# These objects will be created remotely
array = np.zeros((1024, 1024, 1), dtype=np.float32)

solver_1 = Solver1.remote(array)
solver_2 = Solver2.remote()
solver_1

<_TesseraProxy object at 140230614888976, uid=tess-solver1-70c44acf610242cb95993ae2d7eb8bc5, runtime=None, state=pending>

As you can see, the result of calling `remote` is not an instance of the class, but a proxy object.

The mosaic runtime will instantiate `tessera` classes within one of the available workers, and a proxy object will be given to us that points to the remote object.

This proxy allows us to call methods of the remote object as if they were local objects:

In [6]:
# These will run in parallel
# The calls will return immediately by creating a remote
# task
task_1 = solver_1.solve(array)
task_2 = solver_2.solve(array)
task_1

<TaskProxy object at 140230615005008, uid=task-solver1-solve-844e8c30854e4f13a10d5c64a7627f6b, runtime=worker:0:0, state=pending>

WORKER:0:0      Solve 1

WORKER:0:1      Solve 2



Unlike a local method call, calling a remote method will return immediately and will not wait until the work is done. Instead, it will generate a task that the mosaic runtime will pass to the worker who owns the `tessera`, who will queue it for execution.

On our side of the code, the call to the remote method will generate a task proxy that points to its remote counterpart.

Method calls to different `tessera` are executed in parallel, whereas method calls to a specific `tessera` instance are guaranteed to be executed in the order in which they were called.

We can wait for the remote calls to finish by awaiting the proxies:

In [7]:
# Wait until the remote tasks are finished
await task_1
await task_2

WORKER:0:0      Done 1

WORKER:0:1      Done 2



<TaskProxy object at 140230615005456, uid=task-solver2-solve-cba216a41c574f2fa725dc94c1455e64, runtime=worker:0:1, state=done>

The return value of the method calls, if any, will not be transferred back to the user code unless we explicitly request it:

In [8]:
# The results of the tasks stay in the remote worker
# until we request it back
result_1 = await task_1.result()
result_2 = await task_2.result()

print(result_1.shape)
print(result_2.shape)

(1024, 1024, 1)
(1024, 1024, 1)


It is possible to use the return value of a remote method call as an input to another `tessera` by passing the task proxy as an input.

The mosaic runtime will take care of figuring out how to fetch the data that is needed to execute the method.

In [9]:
# These will wait for each other because
# their results depend on each other
task_1 = solver_1.solve(array)
task_2 = solver_2.solve(task_1)
task_1

<TaskProxy object at 140230615027920, uid=task-solver1-solve-0d218afb301943c59f6744f362b94fb4, runtime=worker:0:0, state=pending>

WORKER:0:0      Solve 1



In this case, we only need to wait for the second task to finish because an implicit dependency exists between the two:

In [10]:
# Wait until the remote tasks are finished
# Now we only need to wait for the second task
await task_2

WORKER:0:0      Done 1

WORKER:0:1      Solve 2

WORKER:0:1      Done 2



<TaskProxy object at 140230615028432, uid=task-solver2-solve-2eff683725cf45ce913c31ac3984d489, runtime=worker:0:1, state=done>

We can also create explicit dependencies between two tasks to ensure that they are executed in order:

In [11]:
# These will also wait for each other
task_1 = solver_1.solve_more()
task_2 = solver_2.solve_more(task_1.outputs.done)
task_1

<TaskProxy object at 140230615061392, uid=task-solver1-solve_more-fcc96659d8824299805df79de2003f3e, runtime=worker:0:0, state=pending>

WORKER:0:0      Solve More 1



Again, we only need to wait for the second task to finish.

In [12]:
# Wait until the remote tasks are finished
# Now we only need to wait for the second task
await task_2

WORKER:0:0      Done More 1

WORKER:0:1      Solve More 2

WORKER:0:1      Done More 2



<TaskProxy object at 140230615062032, uid=task-solver2-solve_more-f8705b68f4c74f8c8244cc76c56cfe58, runtime=worker:0:1, state=done>

Before leaving, we should ensure that we tear down the mosaic runtime:

In [13]:
await mosaic.interactive('off')